In [1]:
import xarray as xr
import numpy as np
import rioxarray
import geopandas as gpd
from ipyleaflet import Map, ImageOverlay, WidgetControl, LayersControl, basemaps, GeoJSON, TileLayer, Heatmap
import ipywidgets as widgets
from utils import leaflet_bounds, scalar_to_base64_image
import pandas as pd
from shapely.geometry import Point

In [6]:
# -------------------------------
# Data Paths
# -------------------------------
download_path = "./data_download"
intermediate_path = "./data_intermediate"
# terrain_path = f"{download_path}/data/cv_terrain.tiff"
vector_rivers = f"{download_path}/data/shp/cv_rivers.geojson"
vector_subbasin = f"{download_path}/data/shp/subbasins_cv_clip.geojson"
points_resistivity = f"{download_path}/data/aem/em_resistivity.csv"
metric_zarr_path = f"{intermediate_path}/consolidated_metric_output.zarr"


target_epsg = 4326
center = [37.66335291403956, -120.69523554193438]
zoom = 6
# basemap = basemaps.CartoDB.Positron 
basemap = None
map_width = '500px'
map_height = '800px'

In [3]:
# -------------------------------
# Data Loading
# -------------------------------

# Load vector layers with GeoPandas
rivers = gpd.read_file(vector_rivers)
rivers = rivers.to_crs(epsg=target_epsg)

subbasins = gpd.read_file(vector_subbasin)
subbasins = subbasins.to_crs(epsg=target_epsg)

df = pd.read_csv(points_resistivity)
# Create GeoDataFrame from the UTM coordinates
resistivity_profiles = gpd.GeoDataFrame(
    df,
    geometry=[Point(x, y) for x, y in zip(df['UTMX'], df['UTMY'])],
    crs='EPSG:3310'  # TODO: Confirm EPSG!!!
)
resistivity_profiles = resistivity_profiles.to_crs(f'EPSG:{target_epsg}')
# for now resample all points 
# TODO: Discuss better performance options
resistivity_profiles = resistivity_profiles.sample(10000)

# Load consolidated metric dataset
ds = xr.open_zarr(metric_zarr_path)
ds = ds.transpose('fraction', 'y', 'x')
ds = ds.sortby('y', ascending=False)
ds = ds.sortby('x', ascending=True)
ds.rio.write_crs(3310, inplace=True)
ds_reprojected = ds.rio.reproject(f"EPSG:{target_epsg}")


In [4]:
# Create layers
## Terrain context
# Ocean basemap (includes bathymetry)
l_ocean = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}',
    name='Ocean/Water',
    opacity=1.0
)
# Hillshade with water areas
l_elevation = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}',
    name='Hillshade',
    opacity=1.0
)

# Vector layers
l_rivers = GeoJSON(
    data=rivers.__geo_interface__,
    style={'color': 'blue', 'weight': 1, 'opacity': 0.7},
    name="Rivers"
)

l_subbasins = GeoJSON(
    data=subbasins.__geo_interface__,
    style={'color': 'black', 'weight': 1, 'fill':False},
    name="Subbasins"
)

# Point layers
l_resistivity = GeoJSON(
    data=resistivity_profiles.__geo_interface__,
    point_style={
        'radius': 0.01,
        'color': 'red',
        'fillColor': 'red',
        'fillOpacity': 0.6,
        'weight': 0.1
    },
    name='Resistivity Profiles (subsampled)',
)

l_sediment = GeoJSON(
    data=resistivity_profiles.__geo_interface__,
    point_style={
        'radius': 0.01,
        'color': 'brown',
        'fillColor': 'brown',
        'fillOpacity': 0.6,
        'weight': 0.1
    },
    name='Sediment Type Profiles (subsampled)',
)

# Heatmap layers
resistivity_locations = [[point.y, point.x] for point in resistivity_profiles.geometry]
l_resistivity_heatmap = Heatmap(
    locations=resistivity_locations,
    radius=5,
    blur=2,
    name='Resistivity Heatmap (subsampled)'
)

l_sediment_heatmap = Heatmap(
    locations=resistivity_locations,  # Using same data for now
    radius=10,
    blur=2,
    gradient={0.4: 'blue', 0.6: 'cyan', 0.7: 'lime', 0.8: 'yellow', 1.0: 'red'},
    name='Sediment Heatmap (subsampled)'
)

In [ ]:
#| label: interactive:fig-1

##########
# Figure 1
##########
m = Map(
    center=center, 
    zoom=zoom, 
    # basemap=basemap, #TODO fix
    scroll_wheel_zoom=True,
    layout=widgets.Layout(width=map_width, height=map_height)
)

# Dictionary mapping dropdown options to layer objects
layer_map = {layer.name: layer for layer in [
    l_resistivity,
    l_resistivity_heatmap,
    l_sediment,
    l_sediment_heatmap
]}
init_key = list(layer_map.keys())[0]

m.add_layer(l_ocean)
m.add_layer(l_elevation)
m.add_layer(l_rivers)
m.add_layer(l_subbasins)
m.add_layer(l_resistivity)

# Create dropdown to switch between layers
layer_dropdown = widgets.Dropdown(
    options=list(layer_map.keys()),
    value=init_key,
    description='Data Layer:',
    style={'description_width': 'initial'}
)

# Current active layer
current_layer = layer_map[init_key]

def on_layer_change(change):
    """Handle layer selection change"""
    global current_layer
    new_layer_name = change['new']
    new_layer = layer_map[new_layer_name]
    
    # Remove current layer and add new one
    m.remove_layer(current_layer)
    m.add_layer(new_layer)
    current_layer = new_layer

layer_dropdown.observe(on_layer_change, names='value')

# Add dropdown control to map
widget_control = WidgetControl(widget=layer_dropdown, position='topright')
m.add_control(widget_control)
m.add_control(LayersControl(position='topleft'))
m

Map(center=[37.66335291403956, -120.69523554193438], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [ ]:
import dill
results_rock_physics_grid = dill.load(open(f"{download_path}/data/rock_physics_grid.pik", "rb"))
results_rock_physics_grid

In [ ]:
#| label: nb:map-one




# -------------------------------
# Create map
# -------------------------------
m = Map(
    center=[37.66335291403956, -120.69523554193438], 
    zoom=5, 
    basemap=basemaps.CartoDB.Positron, 
    scroll_wheel_zoom=True
) 

## Terrain context
# Ocean basemap (includes bathymetry)
ocean_layer = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}',
    name='Ocean/Water',
    opacity=1.0
)
m.add_layer(ocean_layer)

# Hillshade with water areas
elevation_tiles = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}',
    name='Hillshade',
    opacity=0.8
)
m.add_layer(elevation_tiles)

# Vector layers
l_rivers = GeoJSON(
    data=rivers.__geo_interface__,
    style={'color': 'blue', 'weight': 1, 'opacity': 0.7},
    name="Rivers"
)

l_subbasins = GeoJSON(
    data=subbasins.__geo_interface__,
    style={'color': 'black', 'weight': 1, 'fill':False},
    name="Subbasins"
)
m.add_layer(l_rivers)
m.add_layer(l_subbasins)

#raster layers

# Start with first dataset
# current_dataset_name = "fraction_coarse"
current_dataset_name = "path_length_norm"
da_scalar = ds_reprojected[current_dataset_name].sel(fraction=0.2).load()
# Add scalar data overlay (will be updated by threshold and dataset selection)
initial_scalar = scalar_to_base64_image(da_scalar, cmap='plasma')
scalar_overlay = ImageOverlay(
    url=initial_scalar, 
    bounds=leaflet_bounds(da_scalar), 
    opacity=1.0, 
    name="Scalar Data"
)
m.add_layer(scalar_overlay)

# -------------------------------
# Controls: Dataset dropdown and threshold slider
# -------------------------------
def update_scalar_overlay():
    """Update the scalar overlay based on current dataset and threshold"""
    threshold = slider.value
    # da_masked = da_scalar.where(da_scalar >= threshold)
    if 'fraction' in da_scalar.dims:
        da_overlay = da_scalar.sel(fraction=threshold)
    else:
        da_overlay = da_scalar

    scalar_overlay.url = scalar_to_base64_image(
        da_overlay, 
        cmap='plasma',
        vmin=float(np.nanmin(da_scalar.values)),
        vmax=float(np.nanmax(da_scalar.values))
    )

def on_dataset_change(change):
    """Handle dataset selection change"""
    global da_scalar, current_dataset_name
    current_dataset_name = change['new']
    da_scalar = ds[current_dataset_name]
    
    # Update the overlay
    update_scalar_overlay()

def on_threshold_change(change):
    """Handle threshold slider change"""
    update_scalar_overlay()

# Dataset dropdown
dropdown = widgets.Dropdown(
    options=list(ds.data_vars),
    value=current_dataset_name,
    description='Dataset:',
    style={'description_width': 'initial'}
)
dropdown.observe(on_dataset_change, names='value')

slider = widgets.SelectionSlider(
    options=ds.fraction.values,
    value=0.1,  # Must be one of the values in options
    description='FCD Threshold',
    style={'description_width': 'initial'}
)
slider.observe(on_threshold_change, names='value')

# Combine controls in a VBox
controls = widgets.VBox([dropdown, slider])
widget_control = WidgetControl(widget=controls, position='topright')
m.add_control(widget_control)

# -------------------------------
# 9. Layer toggle control
# -------------------------------
m.add_control(LayersControl(position='topright'))

m

In [ ]:
leaflet_bounds(ds_reprojected)

In [ ]:
## Terrain mismatch debugging map

import geopandas as gpd
import rioxarray
import ipyleaflet as ipl
import ipywidgets as widgets
from IPython.display import display

target_epsg = 4326

# Load your data
da_terrain = rioxarray.open_rasterio(terrain_path)
da_terrain = da_terrain.rio.reproject(f"EPSG:{target_epsg}")

rivers = gpd.read_file(vector_rivers).to_crs(epsg=target_epsg)
subbasins = gpd.read_file(vector_subbasin).to_crs(epsg=target_epsg)

# Create map
m = ipl.Map(
    center=[37.66, -120.70], 
    zoom=8, 
    basemap=ipl.basemaps.Esri.WorldImagery
)

# Esri Hillshade with water areas
elevation_tiles = ipl.TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}',
    name='Esri Hillshade',
    opacity=0.5
)
m.add_layer(elevation_tiles)

# Esri Ocean basemap (includes bathymetry)
ocean_layer = ipl.TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}',
    name='Ocean/Water',
    opacity=0.5
)
m.add_layer(ocean_layer)

# Vectors
m.add_layer(ipl.GeoJSON(
    data=rivers.__geo_interface__,
    style={'color': 'cyan', 'weight': 2},
    name="Rivers"
))
m.add_layer(ipl.GeoJSON(
    data=subbasins.__geo_interface__,
    style={'color': 'red', 'weight': 2, 'fill': False},
    name="Subbasins"
))

# Opacity sliders
slider_terrain = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.1,
    description='Your Terrain:'
)
slider_topo = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.1,
    description='OpenTopoMap:'
)

widgets.link((slider_terrain, 'value'), (terrain_overlay, 'opacity'))
widgets.link((slider_topo, 'value'), (elevation_tiles, 'opacity'))

controls = widgets.VBox([slider_terrain, slider_topo])
m.add_control(ipl.WidgetControl(widget=controls, position='topright'))
m.add_control(ipl.LayersControl(position='topleft'))

display(m)

In [ ]:
"""
Example: Display GeoPandas data on hover in ipyleaflet GeoJSON layer
"""

import geopandas as gpd
from ipyleaflet import Map, GeoJSON, basemaps
from ipywidgets import HTML, VBox
import json

# Create sample GeoDataFrame (replace with your own data)
# Load Natural Earth data from URL (works with GeoPandas 1.0+)
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world = gpd.read_file(url)

# Select a subset for demonstration
gdf = world[world['CONTINENT'] == 'Europe'].copy()

# Select available columns - using NAME, ISO_A3, and POP_EST which should be present
cols_to_use = ['NAME', 'ISO_A3', 'POP_EST', 'geometry']
gdf = gdf[cols_to_use].copy()
gdf.columns = ['name', 'iso_a3', 'pop_est', 'geometry']

# Create the base map
m = Map(center=(54, 15), zoom=3, basemap=basemaps.OpenStreetMap.Mapnik)

# Convert GeoDataFrame to GeoJSON, adding properties for hover display
def create_geojson_with_properties(gdf, hover_fields):
    """
    Convert GeoDataFrame to GeoJSON with selected properties
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        Input geodataframe
    hover_fields : list
        List of column names to include in properties (displayed on hover)
    
    Returns:
    --------
    dict : GeoJSON dictionary
    """
    geojson = json.loads(gdf.to_json())
    
    # Add desired properties to each feature
    for feature, (idx, row) in zip(geojson['features'], gdf.iterrows()):
        feature['properties'] = {field: str(row[field]) for field in hover_fields}
    
    return geojson

# Specify which columns to display on hover
hover_fields = ['name', 'iso_a3', 'pop_est']

# Create GeoJSON object with hover data
geojson_data = create_geojson_with_properties(gdf, hover_fields)

# Create GeoJSON layer with styling
geo_json = GeoJSON(
    data=geojson_data,
    style={
        'color': 'black',
        'fillColor': '#3388ff',
        'opacity': 0.5,
        'weight': 1,
        'fillOpacity': 0.3,
    },
    hover_style={
        'fillColor': '#ff7800',
        'fillOpacity': 0.7,
    },
)

# Add layer to map
m.add_layer(geo_json)

# Create a tooltip display widget
tooltip_html = HTML(
    value="<b>Hover over a feature to see details</b>",
    description='',
)

# Function to update tooltip on feature interaction
def on_feature_interaction(**kwargs):
    if kwargs.get('type') == 'mouseover':
        properties = kwargs.get('properties', {})
        if properties:
            html_str = "<b>Feature Details:</b><br>"
            for key, value in properties.items():
                html_str += f"<b>{key}:</b> {value}<br>"
            tooltip_html.value = html_str
    elif kwargs.get('type') == 'mouseout':
        tooltip_html.value = "<b>Hover over a feature to see details</b>"

# Connect hover events to tooltip update
geo_json.on_hover(on_feature_interaction)

# Display map and tooltip
output = VBox([tooltip_html, m])
display(output)
# If running as a script, uncomment:
# display(m)

In [ ]:
"""
Debug version: Let's see what events are actually firing
"""
import geopandas as gpd
from ipyleaflet import Map, GeoJSON, basemaps
from ipywidgets import HTML, VBox, Output
import json

# Load Natural Earth data
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world = gpd.read_file(url)

# Select a subset for demonstration
gdf = world[world['CONTINENT'] == 'Europe'].copy()

# Select available columns
cols_to_use = ['NAME', 'ISO_A3', 'POP_EST', 'geometry']
gdf = gdf[cols_to_use].copy()
gdf.columns = ['name', 'iso_a3', 'pop_est', 'geometry']

# Create the base map
m = Map(center=(54, 15), zoom=3, basemap=basemaps.OpenStreetMap.Mapnik)

# Convert GeoDataFrame to GeoJSON
geojson_data = json.loads(gdf.to_json())

print("Number of features:", len(geojson_data['features']))
print("Sample feature properties:", geojson_data['features'][0]['properties'])

# Create GeoJSON layer with styling
geo_json = GeoJSON(
    data=geojson_data,
    style={
        'color': 'black',
        'fillColor': '#3388ff',
        'opacity': 0.5,
        'weight': 1,
        'fillOpacity': 0.3,
    },
    hover_style={
        'fillColor': '#ff7800',
        'fillOpacity': 0.7,
    },
)

# Add layer to map
m.add_layer(geo_json)

# Create debug output
debug_output = Output()
hover_widget = HTML(value="<b>Waiting for hover...</b>")
click_widget = HTML(value="<b>Waiting for click...</b>")

# Counter to see if events are firing at all
event_counter = {'hover': 0, 'click': 0}

# Function to handle hover events
def handle_hover(**kwargs):
    event_counter['hover'] += 1
    with debug_output:
        print(f"\n=== HOVER EVENT #{event_counter['hover']} ===")
        print(f"Event type: {kwargs.get('type')}")
        print(f"All kwargs keys: {list(kwargs.keys())}")
        print(f"Full kwargs: {kwargs}")
    
    if kwargs.get('type') == 'mouseover':
        feature = kwargs.get('feature')
        if feature:
            props = feature.get('properties', {})
            name = props.get('name', 'Unknown')
            hover_widget.value = f"<b style='color: green;'>HOVER: {name}</b>"
        else:
            hover_widget.value = "<b style='color: red;'>HOVER: No feature found</b>"

# Function to handle click events
def handle_click(**kwargs):
    event_counter['click'] += 1
    with debug_output:
        print(f"\n=== CLICK EVENT #{event_counter['click']} ===")
        print(f"Event type: {kwargs.get('type')}")
        print(f"All kwargs keys: {list(kwargs.keys())}")
        print(f"Full kwargs: {kwargs}")
    
    if kwargs.get('type') == 'click':
        feature = kwargs.get('feature')
        if feature:
            props = feature.get('properties', {})
            iso = props.get('iso_a3', 'N/A')
            click_widget.value = f"<b style='color: blue;'>CLICK: ISO={iso}</b>"
        else:
            click_widget.value = "<b style='color: red;'>CLICK: No feature found</b>"

# Connect events
geo_json.on_hover(handle_hover)
geo_json.on_click(handle_click)

# Display everything
output = VBox([
    hover_widget,
    click_widget,
    HTML(value="<hr><b>Debug Console (check for event data):</b>"),
    debug_output,
    HTML(value="<hr>"),
    m
])
display(output)

print("\n✅ Setup complete. Now try hovering and clicking on countries.")
print("Check the debug console above to see what data is being passed.")

In [ ]:
"""
Working version: Display GeoPandas data on hover and click in ipyleaflet GeoJSON layer
"""
import geopandas as gpd
from ipyleaflet import Map, GeoJSON, basemaps
from ipywidgets import HTML, VBox
import json

# Load Natural Earth data
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world = gpd.read_file(url)

# Select a subset for demonstration
gdf = world[world['CONTINENT'] == 'Europe'].copy()

# Select available columns
cols_to_use = ['NAME', 'ISO_A3', 'POP_EST', 'geometry']
gdf = gdf[cols_to_use].copy()
gdf.columns = ['name', 'iso_a3', 'pop_est', 'geometry']

# Create the base map
m = Map(center=(54, 15), zoom=3, basemap=basemaps.OpenStreetMap.Mapnik)

# Convert GeoDataFrame to GeoJSON
geojson_data = json.loads(gdf.to_json())

# Create GeoJSON layer with styling
geo_json = GeoJSON(
    data=geojson_data,
    style={
        'color': 'black',
        'fillColor': '#3388ff',
        'opacity': 0.5,
        'weight': 1,
        'fillOpacity': 0.3,
    },
    hover_style={
        'fillColor': '#ff7800',
        'fillOpacity': 0.7,
    },
)

# Add layer to map
m.add_layer(geo_json)

# Create info widgets
hover_widget = HTML(
    value="<div style='padding: 10px; background-color: #f0f0f0; border-radius: 5px;'><b>Hover over a country</b></div>",
)

click_widget = HTML(
    value="<div style='padding: 10px; background-color: #e0e0ff; border-radius: 5px;'><b>Click on a country</b></div>",
)

# Function to handle hover events
def handle_hover(**kwargs):
    event_type = kwargs.get('event')  # FIXED: changed from 'type' to 'event'
    
    if event_type == 'mouseover':
        feature = kwargs.get('feature')
        if feature and 'properties' in feature:
            props = feature['properties']
            name = props.get('name', 'Unknown')
            # Display ONLY the country name on hover
            hover_widget.value = f"<div style='padding: 10px; background-color: #fff3cd; border-radius: 5px;'><b>🏴 Hovering:</b> {name}</div>"
    
    elif event_type == 'mouseout':
        hover_widget.value = "<div style='padding: 10px; background-color: #f0f0f0; border-radius: 5px;'><b>Hover over a country</b></div>"

# Function to handle click events
def handle_click(**kwargs):
    event_type = kwargs.get('event')  # FIXED: changed from 'type' to 'event'
    
    if event_type == 'click':
        feature = kwargs.get('feature')
        if feature and 'properties' in feature:
            props = feature['properties']
            iso_code = props.get('iso_a3', 'N/A')
            pop = props.get('pop_est', 'N/A')
            
            # Format population
            if pop != 'N/A' and pop is not None:
                pop = f"{int(pop):,}"
            
            # Display ISO code and population on click
            click_widget.value = f"""
            <div style='padding: 10px; background-color: #d4edda; border-radius: 5px;'>
                <b>🗺️ Clicked Country:</b><br>
                <b>ISO Code:</b> {iso_code}<br>
                <b>Population:</b> {pop}
            </div>
            """

# Connect events
geo_json.on_hover(handle_hover)
geo_json.on_click(handle_click)

# Display map and info widgets in a vertical layout
output = VBox([hover_widget, click_widget, m])
display(output)

In [ ]:
"""
Example: Display tooltip on map with GeoPandas data on hover
"""
import geopandas as gpd
from ipyleaflet import Map, GeoJSON, basemaps, Popup, Marker
from ipywidgets import HTML
import json

# Load Natural Earth data
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world = gpd.read_file(url)

# Select a subset for demonstration
gdf = world[world['CONTINENT'] == 'Europe'].copy()

# Select available columns
cols_to_use = ['NAME', 'ISO_A3', 'POP_EST', 'geometry']
gdf = gdf[cols_to_use].copy()
gdf.columns = ['name', 'iso_a3', 'pop_est', 'geometry']

# Create the base map
m = Map(center=(54, 15), zoom=3, basemap=basemaps.OpenStreetMap.Mapnik)

# Convert GeoDataFrame to GeoJSON
geojson_data = json.loads(gdf.to_json())

# Create GeoJSON layer with styling
geo_json = GeoJSON(
    data=geojson_data,
    style={
        'color': 'black',
        'fillColor': '#3388ff',
        'opacity': 0.5,
        'weight': 1,
        'fillOpacity': 0.3,
    },
    hover_style={
        'fillColor': '#ff7800',
        'fillOpacity': 0.7,
    },
)

# Add layer to map
m.add_layer(geo_json)

# Store current popup to remove it later
current_popup = None

# Function to handle hover events and show tooltip
def handle_hover(**kwargs):
    global current_popup
    
    event_type = kwargs.get('event')
    
    if event_type == 'mouseover':
        feature = kwargs.get('feature')
        coordinates = kwargs.get('coordinates')  # [lat, lon]
        
        if feature and 'properties' in feature and coordinates:
            props = feature['properties']
            name = props.get('name', 'Unknown')
            iso = props.get('iso_a3', 'N/A')
            pop = props.get('pop_est', 'N/A')
            
            # Format population
            if pop != 'N/A' and pop is not None:
                pop_formatted = f"{int(pop):,}"
            else:
                pop_formatted = 'N/A'
            
            # Create HTML content for the popup
            popup_content = HTML()
            popup_content.value = f"""
            <div style='min-width: 200px;'>
                <h3 style='margin: 0 0 10px 0; color: #2c3e50;'>{name}</h3>
                <table style='width: 100%; border-collapse: collapse;'>
                    <tr>
                        <td style='padding: 5px; font-weight: bold;'>ISO Code:</td>
                        <td style='padding: 5px;'>{iso}</td>
                    </tr>
                    <tr style='background-color: #f8f9fa;'>
                        <td style='padding: 5px; font-weight: bold;'>Population:</td>
                        <td style='padding: 5px;'>{pop_formatted}</td>
                    </tr>
                </table>
            </div>
            """
            
            # Remove previous popup if exists
            if current_popup and current_popup in m.layers:
                m.remove_layer(current_popup)
            
            # Create and add new popup at the hover location
            # Note: coordinates are [lat, lon] from ipyleaflet
            popup = Popup(
                location=coordinates,
                child=popup_content,
                close_button=False,
                auto_close=True,
                close_on_escape_key=False
            )
            m.add_layer(popup)
            current_popup = popup
    
    elif event_type == 'mouseout':
        # Remove popup when mouse leaves
        if current_popup and current_popup in m.layers:
            m.remove_layer(current_popup)
            current_popup = None

# Function to handle click events (optional - for permanent markers)
def handle_click(**kwargs):
    event_type = kwargs.get('event')
    
    if event_type == 'click':
        feature = kwargs.get('feature')
        coordinates = kwargs.get('coordinates')
        
        if feature and 'properties' in feature and coordinates:
            props = feature['properties']
            name = props.get('name', 'Unknown')
            
            # Create a marker that stays on the map
            marker_content = HTML()
            marker_content.value = f"<b>📍 {name}</b>"
            
            marker = Marker(location=coordinates)
            marker.popup = marker_content
            m.add_layer(marker)

# Connect events
geo_json.on_hover(handle_hover)
geo_json.on_click(handle_click)

# Display map
display(m)

In [ ]:
geojson_data